In [1]:
import son_main_script as soon
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pymoo.indicators.hv import Hypervolume
import json
import os
import plotly.graph_objects as go
import numpy as np

In [2]:
# load dataset

from cProfile import label
from re import X
from matplotlib import axis

from son_pymoo import ObjectiveEnum


def sign_objective(value: float, key: str):
    if(key == ObjectiveEnum.AVG_DL_RATE.name or 
       key == ObjectiveEnum.AVG_RSSI.name or
       key == ObjectiveEnum.AVG_SINR.name or
       key == ObjectiveEnum.ENERGY_EFFICIENCY.name):
        return -value
    else:
        return value

network_name = "hetNet"
param_configuration_list =["algorithm_config_1", "algorithm_config_2", "algorithm_config_3", "algorithm_config_4"]
results_objectives_list = []
results_objective_spaces_list = []
result_dict_object = {"optimization_objectives": []}
ref_points = []
histories = []
for configuration_name in param_configuration_list:
    results_directory = "./datastore/{network_name}/{configuration_name}/".format(network_name=network_name, configuration_name= configuration_name)

    ## read json file 
    with open(results_directory + "/objectives_result.json", mode="r",encoding="utf-8") as openfile:
        result_dict_object = json.load(openfile)
        ref_points = [1.1 for  x in result_dict_object["optimization_objectives"]]

        results_objectives_list.append(np.array(
        [[sign_objective(y[1][key], key) for key in result_dict_object["optimization_objectives"]]
        for y in [item for item in result_dict_object["results"]]]))
        histories.append({"name": configuration_name, "history": result_dict_object["history"]})
        
        results_objective_spaces_list.append(np.array(result_dict_object["objectiveSpace"]))

## prepare diagrams
fig2 = go.Figure()
fig2.update_layout(
    title="pareto fronts",
    xaxis_title=result_dict_object["optimization_objectives"][0],
    yaxis_title=result_dict_object["optimization_objectives"][1],
    legend_title="configuration names",
)
fig1 = go.Figure()
fig1.update_layout(
    title="objective results",
    xaxis_title=result_dict_object["optimization_objectives"][0],
    yaxis_title=result_dict_object["optimization_objectives"][1],
    legend_title="configuration names",
)
fig3 = go.Figure()
fig3.update_layout(
    title="Convergence",
    xaxis_title="Function Evaluations",
    yaxis_title="Hypervolume",
    legend_title="configuration names"
)
## normalize each solution and calculate hyper volume and add dataj to diagrams
## since the ideal and nadir points and pareto front are not known we approximate them
combined_result_list = []
for _, solution in enumerate(results_objectives_list):
    for _, solution_candidate in enumerate(solution):
        combined_result_list.append(solution_candidate)
combined_result_list = np.array(combined_result_list)
approx_ideal = combined_result_list.min(axis=0)
approx_nadir  = combined_result_list.max(axis=0)
hv = Hypervolume(ref_point=np.array(ref_points),
                     norm_ref_point=False,
                     zero_to_one=True,
                     ideal=approx_ideal,
                     nadir=approx_nadir)

for index, solution in enumerate(results_objectives_list):
    normalized_solution = np.array((solution - approx_ideal) / (approx_nadir - approx_ideal))
    print("hyper volume")
    print(hv(normalized_solution))
    print(hv(solution))
    fig2.add_trace(go.Scatter(x=normalized_solution[:,0], y=normalized_solution[:,1],
                    mode='markers',
                    name=param_configuration_list[index] ))
    
    fig1.add_trace(go.Scatter(x=solution[:,0], y=solution[:,1],
                mode='markers',
                name=param_configuration_list[index] ))

## add data to convergence diagram
## since the ideal and nadir points and pareto front are not known we approximate them
combined_result_list = []
for _, solution in enumerate(results_objective_spaces_list):
    for _, solution_candidate in enumerate(solution):
        combined_result_list.append(solution_candidate)
combined_result_list = np.array(combined_result_list)
approx_ideal = combined_result_list.min(axis=0)
approx_nadir  = combined_result_list.max(axis=0)
hv = Hypervolume(ref_point=np.array(ref_points),
                     norm_ref_point=False,
                     zero_to_one=True,
                     ideal=approx_ideal,
                     nadir=approx_nadir)


for _, history  in enumerate(histories):
    hv_list = [hv(np.array(solution)) for solution in history["history"]["objective_space_opt"]]
    
    fig3.add_trace(
        go.Scatter(
            x=history["history"]["n_evals"],
            y=hv_list,
            mode='lines+markers',
            name=history["name"]
        )
    )

fig2.show()
fig1.show()
fig3.show()



hyper volume
0.0
0.5306596812934345
hyper volume
0.0
0.8571328805252257
hyper volume
0.0
0.88987993003311
hyper volume
0.0
0.31405342415762455
